#### 1. Extract Information from 165 PDFs from Mental Health Atlas 2020

In [65]:
import os
import pandas as pd

def extract_information_from_pdf(pdf_path):
    extracted_information = {}
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)

        first_page_text = ""
        for page_num in range(reader.numPages):
            page = reader.getPage(page_num)
            first_page_text += page.extractText()

        keywords = ['expenditure[']
        keywords_8 = 'output in total research'
        for keyword in keywords:
            start_index = first_page_text.find(keyword)
            if start_index != -1:
                end_index = first_page_text.find("]", start_index) 
                value = first_page_text[start_index+len(keyword):end_index].strip()
                extracted_information[keyword] = value

        start_index = first_page_text.find(keywords_8)    
        if start_index != -1:
                newline_count = 0
                index = start_index
                while newline_count < 8 and index < len(first_page_text):
                    if first_page_text[index] == '\n':
                        newline_count += 1
                    index += 1
        else: return None
        if newline_count == 8:
                    start_index = index
                    end_index = first_page_text.find('\n', start_index)
                    if end_index != -1:
                        value = first_page_text[start_index:end_index].strip()
                        extracted_information[keywords_8] = value
        else: return None 

    # return first_page_text
    return extracted_information

folder_path = "pdf_files"

extracted_data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        extracted_info = extract_information_from_pdf(pdf_path)
        extracted_data.append(extracted_info)

extract_information_from_pdf(pdf_path)


Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'23' b'0'
Superfluous whitespace found in object header b'34' b'0'
Superfluous whitespace found in object header b'22' b'0'
Superfluous whitespace found in object header b'11' b'0'
Superfluous whitespace found in object header b'10' b'0'
Superfluous whitespace found in object header b'9' b'0'
Superfluous whitespace found in object header b'19' b'0'
Superfluous whitespace found in object header b'18' b'0'
Superfluous whitespace found in object header b'17' b'0'
Superfluous whitespace found in object header b'21' b'0'
Superfluous whitespace found in object header b'20' b'0'
Superfluous whitespace found in object header b'33' b'0'
Superfluous whitespace found in object header b'31' b'0'
Superfluous whitespace found in object header b'30' b'0'
Superfluous whitespace found in obj

{'expenditure[': 'Estonia', 'output in total research': '929 70.08'}

In [66]:
filtered_list = [x for x in extracted_data if x is not None]

In [67]:
df = pd.DataFrame(filtered_list)

#### 2. Clean the dataframe with the extracted data

In [68]:
df = df.rename(columns={'expenditure[': 'country'})

In [50]:
df

,country,output in total research
0,Egypt,5 854 5.83
1,Japan,141 986 111.92
2,Cabo Verde,76 13.82
3,Costa Rica,7 410 146.80
4,Syrian Arab Republic,7 259 42.52
...,...,...
159,Spain,4 026 8.61
160,Argentina,144 399 322.46
161,Jamaica,601 20.38
162,Madagascar,388 1.44


In [69]:
# Strip out everything to the left of the last whitespace
df['total'] = df['output in total research'].str.rsplit(n=1).str[0]
df['per_100000_pop'] = df['output in total research'].str.rsplit(n=1).str[-1]

In [59]:
df = df.rename(columns = {'test':'total', 'per': 'per_100000_pop'})

In [71]:
df.drop(columns=['output in total research'], axis=1, inplace=True)

In [72]:
df

,country,total,per_100000_pop
0,Egypt,5 854,5.83
1,Japan,141 986,111.92
2,Cabo Verde,76,13.82
3,Costa Rica,7 410,146.80
4,Syrian Arab Republic,7 259,42.52
...,...,...,...
159,Spain,4 026,8.61
160,Argentina,144 399,322.46
161,Jamaica,601,20.38
162,Madagascar,388,1.44


#### 3. Convert to CSV for further cleaning

In [73]:
import csv

In [ ]:
df.to_csv('lana.csv', index=False)